# The travelling foodie problem.

Okay, I must admit that when I came up with this idea I was feeling really hungry! Being in a different country makes you miss your home town food. So I thought, if I had only one day in the best city in India (I know you might disagree but anyway...), which is Pune of course, which restaurants would I visit in the least amount of time.
Instead of asking people what their opinions are on this, let's take a simple approach by scraping data from a website and let's rank restaurants by average reviews per number of reviews a particular restaurant received.

In [6]:
import requests
import json
from APIKeys import *

base_url = 'https://developers.zomato.com/api/v2.1/search?entity_id=5&entity_type=city&sort=rating&order=desc&start='
init = 0
headers = {'Accept': 'application/json', 'user-key': zkey}
restsList = []

r = json.loads(requests.get(base_url + str(init), headers = headers).content.decode("utf-8"))
restsList.append(r['restaurants'])

#print(restsList)
dictList = []
for i in restsList:
    for j in i:
        dictList.append(j['restaurant'])
#for rests in r['restaurants']:
#    print(rests['restaurant']['name'])
import pandas as pd
toprest = pd.DataFrame(dictList).drop('apikey',axis = 1)
toprest['wrating'] = list(map(lambda rate: float(rate['aggregate_rating']),toprest['user_rating']))

https://www.quora.com/How-does-zomato-rating-work

In [7]:
toprest.sort_values('wrating',ascending = False,inplace = True)
toprest['longitude'] = list(map(lambda rate: float(rate['longitude']),toprest['location']))
toprest['latitude'] = list(map(lambda rate: float(rate['latitude']),toprest['location']))
toprest['address'] = list(map(lambda rate: rate['address'],toprest['location']))

In [8]:
toprest[["name","longitude","latitude","address","wrating"]].head()

,name,longitude,latitude,address,wrating
0,Chili's American Grill & Bar,73.917050,18.562406,"UG 49, Phoenix Market City, Nagar Road, Viman ...",4.9
2,Arth,73.906198,18.539464,"The Westin Pune, 36/-3B, Pingale Wasti, Korega...",4.9
3,Henny's Gourmet,73.902032,18.481553,Girme Heights. Salunkevihar Road. Next to ICIC...,4.9
4,Le Flamington,73.905952,18.553554,"Plot 88, Survey 2016, East Avenue, Kalyani Nag...",4.9
5,La Kheer Deli,73.809039,18.556957,"LKD, outside Starbucks, ITI road, Aundh-07.",4.9


In [21]:
#Starting location
origlong = '73.908040'
origlatitude  = '18.578980'
locOrigin = origlatitude + ',' + origlong


In [22]:
#Building the query to get the distances from google maps
Gquery = 'https://maps.googleapis.com/maps/api/distancematrix/json?&key='
Gquery = 'https://maps.googleapis.com/maps/api/distancematrix/json?&key='

origins = locOrigin + "|" + "|".join((toprest['latitude'].apply(str) +',' + toprest['longitude'].apply(str)).tolist()[:3])


DistMat = json.loads(requests.get(Gquery+ GApiKey+ '&origins=' + origins + '&destinations=' + origins).content.decode("utf-8"))

In [23]:
DistMat['destination_addresses']

['Pune Airport Departure, New Airport Rd, Pune International Airport Area, Sanjay Park, Pune, Maharashtra 411014, India',
 '207, Viman Nagar Rd, Clover Park, Viman Nagar, Pune, Maharashtra 411014, India',
 'N Main Rd, Koregaon Park Annexe, Mundhwa, Pune, Maharashtra 411001, India',
 'Empire Landmark, Salunke Vihar Road, Oxford Village, Kedari Nagar, Wanowrie, Pune, Maharashtra 411040, India']

In [99]:
DistMat['rows'][0]['elements'][2]

{'distance': {'text': '3.5 km', 'value': 3480},
 'duration': {'text': '12 mins', 'value': 693},
 'status': 'OK'}

In [65]:
matDistances = []
for rows in DistMat['rows']:
    rowsList = []
    for elements in rows['elements']:
        #print(elements['distance']['value'])
        rowsList.append(elements['distance']['value'])
    matDistances.append(rowsList)

In [27]:
matDistances

[[0, 2966, 4961, 13462],
 [2949, 0, 4400, 13209],
 [8498, 7972, 0, 10731],
 [12086, 11559, 7919, 0]]

In [64]:
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

# Distance callback
def create_distance_callback(dist_matrix):
# Create a callback to calculate distances between cities.

    def distance_callback(from_node, to_node):
        return int(dist_matrix[from_node][to_node])

    return distance_callback


# Cities
lst = toprest.head(3)['name'].tolist()
lst.insert( 0,"Starting Location") # Adding starting location to the list of places to visit.
city_names = lst



# Distance matrix
dist_matrix = matDistances
tsp_size = len(toprest.head(3)['name'].tolist()) + 1 # Adding starting location
num_routes = 1
depot = 0

# Create routing model
if tsp_size > 0:
    routing = pywrapcp.RoutingModel(tsp_size, num_routes, depot)
    search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
    # Create the distance callback.
    dist_callback = create_distance_callback(dist_matrix)
    routing.SetArcCostEvaluatorOfAllVehicles(dist_callback)
    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)
    if assignment:
      # Solution distance.
        print ("Total distance: " + str(assignment.ObjectiveValue()) + " mtrs\n")
      # Display the solution.
      # Only one route here; otherwise iterate from 0 to routing.vehicles() - 1
        route_number = 0
        index = routing.Start(route_number) # Index of the variable for the starting node.
        route = ''
        while not routing.IsEnd(index):
        # Convert variable indices to node indices in the displayed route.
            route += str(city_names[routing.IndexToNode(index)]) + ' -> '
            index = assignment.Value(routing.NextVar(index))
        route += str(city_names[routing.IndexToNode(index)])
        print ("Route:\n\n" + route)
    else:
        print('No solution found.')
else:
    print ('Specify an instance greater than 0.')



Total distance: 30183 mtrs

Route:

Starting Location -> Chili's American Grill & Bar -> Arth -> Henny's Gourmet -> Starting Location
